# Steps

merge
- merge CHR with fips code using 'cbsacode' column
- merge SMART with CHR
- select columns
- aggregate the data

In [87]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [88]:
data_dir = "..\\data\\"

In [89]:
chr = pd.read_csv(f"{data_dir}combinedCHR_selectedColumns_calculated.csv")[1:]

In [90]:
len(chr.fipscode.unique())

3201

In [91]:
smart_cbsa = pd.read_csv(f"{data_dir}20241030depression.csv", usecols=lambda column: column != 'Unnamed: 0') ## change

In [92]:
smart_cbsa.shape

(1772, 27)

In [93]:
chr.head(3)

,%_65_and_older,%_american_indian_&_alaska_native,%_asian,%_below_18_years_of_age,%_female,%_hispanic,%_native_hawaiian/other_pacific_islander,%_not_proficient_in_english,%_rural,adult_obesity,adult_population,adult_population18_65,adult_smoking,children_eligible_for_free_or_reduced_price_lunch,children_in_poverty,children_in_single-parent_households,county,county_ranked,countycode,diabetes_prevalence,driving_alone_to_work,excessive_drinking,fipscode,high_school_graduation,hiv_prevalence,homicides,low_birthweight,mammography_screening,median_household_income,mental_health_providers,motor_vehicle_crash_deaths,physical_inactivity,poor_mental_health_days,poor_or_fair_health,poor_physical_health_days,population,population_below_18,premature_death,preventable_hospital_stays,primary_care_physicians,ratio_of_population_to_mental_health_providers.,ratio_of_population_to_primary_care_physicians.,sexually_transmitted_infections,some_college,state,statecode,teen_births,total_65_and_older,total_adult_obesity,total_adult_smoking,total_below_18_years_of_age,total_children_in_poverty,total_children_in_single-parent_households,total_excessive_drinking,total_female,total_homicides,total_mental_health_providers,total_motor_vehicle_crash_deaths,total_not_proficient_in_english,total_physical_inactivity,total_poor_mental_health_days,total_poor_or_fair_health,total_poor_physical_health_days,total_premature_death,total_primary_care_physicians,total_rural,total_sexually_transmitted_infections,total_uninsured_adults,total_violent_crime,unemployment,uninsured_adults,violent_crime,year
1,0.138,0.005,0.011,0.239,0.516,0.031,0.0,0.020,0.446,0.319,3583326.788,2933525.084,0.236,0.512,0.221,0.362,Alabama,NaN,0,0.119,0.835,0.126,1000.0,0.671,NaN,NaN,0.1021,0.625100,42586.0,0.000198,25.6,0.305,4.1,0.201,4.2,4708708.0,1125381.212,10189.2,87.9589,0.000798,5051.0,1253.675690,535.0,0.551,AL,1,53.2,649801.704,1.143081e+06,845665.121968,1125381.212,248709.247852,407387.998744,451499.175288,2429693.328,NaN,0.009323,1205.429248,94174.160,1.092915e+06,643523.426667,720248.684388,659219.120000,479779.675536,0.037559,2100083.768,25191.587800,501632.789364,20336.909852,0.101,0.171,431.9,2011.0
2,0.116,0.005,0.006,0.278,0.514,0.022,0.0,0.008,0.448,0.315,36645.832,30758.136,0.274,0.288,0.149,0.297,Autauga County,1.0,1,0.114,0.855,0.141,1001.0,0.740,169.9,NaN,0.0945,0.695122,51622.0,0.000020,28.3,0.327,4.1,0.258,5.2,50756.0,14110.168,9967.4,78.8536,0.000437,50354.0,2288.818182,307.8,0.554,AL,1,52.0,5887.696,1.154344e+04,10040.957968,14110.168,2102.415032,4190.719896,5167.062312,26088.584,NaN,0.000010,14.363948,406.048,1.198319e+04,6936.653333,9454.624656,8797.706667,5059.053544,0.000222,22738.688,156.226968,4767.511080,130.087628,0.089,0.155,256.3,2011.0
3,0.170,0.005,0.006,0.231,0.510,0.033,0.0,0.018,0.542,0.262,138326.182,107746.922,0.219,0.289,0.151,0.289,Baldwin County,1.0,3,0.098,0.829,0.194,1003.0,0.681,176.1,NaN,0.0882,0.724967,51957.0,0.000165,23.2,0.252,4.1,0.128,3.4,179878.0,41551.818,8321.8,63.9873,0.000817,6076.0,1223.694444,226.4,0.615,AL,1,49.6,30579.260,3.624146e+04,30293.433858,41551.818,6274.324518,12008.475402,26835.279308,91737.780,NaN,0.000297,41.731696,3237.804,3.485820e+04,24583.326667,17705.751296,20386.173333,14969.087404,0.001470,97493.876,407.243792,22519.106698,350.222466,0.088,0.209,194.7,2011.0


In [94]:
smart_cbsa.head(3)

,Year,Locationabbr,Locationdesc,Response,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,LocationID,QuestionID,RESPONSEID,GeoLocation,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,depression_twitter19,CENSUS2019POP,depression_twitter10,sample_size_twitter10,CENSUS2010POP
0,2011,10100.0,"Aberdeen, SD Micropolitan Statistical Area",Yes,88.0,16.75,11.83,21.67,50.0,%,Crude Prevalence,NaN,NaN,10100.0,ADDEPEV2,RESP046,"(45.5222131, -98.7041456)",10100,10100.0,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,NaN,NaN,NaN
1,2011,10420.0,"Akron, OH Metropolitan Statistical Area",Yes,129.0,16.18,12.24,20.12,50.0,%,Crude Prevalence,NaN,NaN,10420.0,ADDEPEV2,RESP046,"(41.1466315, -81.3501066)",10420,10420.0,184.0,"Akron, OH",Metropolitan Statistical Area,NaN,NaN,0.205128,39.0,703200.0
2,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10500,NaN,NaN,NaN,NaN,NaN,NaN,0.266667,15.0,153857.0


In [95]:
len(smart_cbsa.cbsacode.unique())

252

In [96]:
smart_cbsa.groupby("Year").size()

Year
2011    216
2012    187
2013    148
2014    134
2015    133
2016    144
2017    139
2018    137
2019    152
2020    122
2021    126
2022    134
dtype: int64

## Merge

In [97]:
print(smart_cbsa.shape, chr.shape)

(1772, 27) (38322, 73)


In [98]:
chr.fipscode.nunique()

3200

In [99]:
cbsa = pd.read_csv(f"{data_dir}cbsa2fipsxw.csv")
cbsa['metropolitandivisioncode'] = cbsa['metropolitandivisioncode'].fillna(cbsa['cbsacode']) # brutal force, alternatively. could dropna()
cbsa['fipscode'] = cbsa.fipsstatecode.astype(str) + cbsa.fipscountycode.astype(str).str.zfill(3)

cbsa = cbsa[["fipscode", "cbsacode"]] #, "metropolitandivisioncode","csacode", "cbsatitle", "metropolitanmicropolitanstatis", "fipscode"]]
cbsa.head(3)

,fipscode,cbsacode
0,1001,33860
1,1003,19300
2,1005,21640


In [100]:
print("chr.shape", chr.shape)
chr = chr.dropna(subset=['fipscode']) ## brutal
print("chr.shape", chr.shape)

print(cbsa.shape, chr.shape)

chr['fipscode'] = chr['fipscode'].astype(int)
cbsa['fipscode'] = cbsa['fipscode'].astype(int)

chr_cbsa = pd.merge(
    cbsa, chr, 
    on='fipscode', 
    how='inner' ##
)

chr_cbsa.shape

chr.shape (38322, 73)
chr.shape (38321, 73)
(1915, 2) (38321, 73)


(22020, 74)

In [101]:
# sanity check
chr['fipscode'] = chr['fipscode'].astype(int)
cbsa['fipscode'] = cbsa['fipscode'].astype(int)

merged_df = pd.merge(
    chr, cbsa, 
    on='fipscode', 
    how='left',
    indicator=True 
)

print("merged_df.shape:", merged_df.shape)

left_only_rows = merged_df[merged_df['_merge'] == 'left_only']

print("Left-only rows in 'chr':")
print(len(left_only_rows))


merged_df.shape: (38321, 75)
Left-only rows in 'chr':
16301


In [102]:
print(chr_cbsa.shape, smart_cbsa.shape)

(22020, 74) (1772, 27)


In [103]:
smart_cbsa.cbsacode

0       10100
1       10420
2       10500
3       10580
4       10740
        ...  
1767    48140
1768    48300
1769    48620
1770    49340
1771    49660
Name: cbsacode, Length: 1772, dtype: int64

In [104]:
print("Columns in smart_cbsa:", smart_cbsa.columns)
print("Columns in chr:", chr_cbsa.columns)


Columns in smart_cbsa: Index(['Year', 'Locationabbr', 'Locationdesc', 'Response', 'Sample_Size',
       'Data_value', 'Confidence_limit_Low', 'Confidence_limit_High',
       'Display_order', 'Data_value_unit', 'Data_value_type',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'LocationID',
       'QuestionID', 'RESPONSEID', 'GeoLocation', 'cbsacode',
       'metropolitandivisioncode', 'csacode', 'cbsatitle',
       'metropolitanmicropolitanstatis', 'depression_twitter19',
       'CENSUS2019POP', 'depression_twitter10', 'sample_size_twitter10',
       'CENSUS2010POP'],
      dtype='object')
Columns in chr: Index(['fipscode', 'cbsacode', '%_65_and_older',
       '%_american_indian_&_alaska_native', '%_asian',
       '%_below_18_years_of_age', '%_female', '%_hispanic',
       '%_native_hawaiian/other_pacific_islander',
       '%_not_proficient_in_english', '%_rural', 'adult_obesity',
       'adult_population', 'adult_population18_65', 'adult_smoking',
       'children_eligible

In [105]:
data = pd.merge(
    smart_cbsa, chr_cbsa,
    left_on=['cbsacode', 'Year'],
    right_on=['cbsacode', 'year'],
    how='left'
)
data.head(3)

,Year,Locationabbr,Locationdesc,Response,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,LocationID,QuestionID,RESPONSEID,GeoLocation,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,depression_twitter19,CENSUS2019POP,depression_twitter10,sample_size_twitter10,CENSUS2010POP,fipscode,%_65_and_older,%_american_indian_&_alaska_native,%_asian,%_below_18_years_of_age,%_female,%_hispanic,%_native_hawaiian/other_pacific_islander,%_not_proficient_in_english,%_rural,adult_obesity,adult_population,adult_population18_65,adult_smoking,children_eligible_for_free_or_reduced_price_lunch,children_in_poverty,children_in_single-parent_households,county,county_ranked,countycode,diabetes_prevalence,driving_alone_to_work,excessive_drinking,high_school_graduation,hiv_prevalence,homicides,low_birthweight,mammography_screening,median_household_income,mental_health_providers,motor_vehicle_crash_deaths,physical_inactivity,poor_mental_health_days,poor_or_fair_health,poor_physical_health_days,population,population_below_18,premature_death,preventable_hospital_stays,primary_care_physicians,ratio_of_population_to_mental_health_providers.,ratio_of_population_to_primary_care_physicians.,sexually_transmitted_infections,some_college,state,statecode,teen_births,total_65_and_older,total_adult_obesity,total_adult_smoking,total_below_18_years_of_age,total_children_in_poverty,total_children_in_single-parent_households,total_excessive_drinking,total_female,total_homicides,total_mental_health_providers,total_motor_vehicle_crash_deaths,total_not_proficient_in_english,total_physical_inactivity,total_poor_mental_health_days,total_poor_or_fair_health,total_poor_physical_health_days,total_premature_death,total_primary_care_physicians,total_rural,total_sexually_transmitted_infections,total_uninsured_adults,total_violent_crime,unemployment,uninsured_adults,violent_crime,year
0,2011,10100.0,"Aberdeen, SD Micropolitan Statistical Area",Yes,88.0,16.75,11.83,21.67,50.0,%,Crude Prevalence,NaN,NaN,10100.0,ADDEPEV2,RESP046,"(45.5222131, -98.7041456)",10100,10100.0,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,NaN,NaN,NaN,46013.0,0.166,0.031,0.007,0.224,0.514,0.013,0.001,0.010,0.299,0.295,27318.304,21474.440,0.191,0.178,0.126,0.248,Brown County,1.0,13.0,0.076,0.801,0.203,0.799,NaN,2.0,0.0602,0.708861,43140.0,0.000228,11.4,0.278,2.4,0.126,3.0,35204.0,7885.696,5179.1,51.3919,0.000942,4378.0,1061.333333,224.7,0.704,SD,46.0,28.5,5843.864,8058.899680,5217.796064,7885.696,993.597696,1955.652608,5545.615712,18094.856,0.70408,0.000080,4.013256,352.04,7594.488512,2816.320000,3442.106304,3520.400000,1823.250364,0.000332,10525.996,79.103388,2899.049400,NaN,0.035,0.135,NaN,2011.0
1,2011,10100.0,"Aberdeen, SD Micropolitan Statistical Area",Yes,88.0,16.75,11.83,21.67,50.0,%,Crude Prevalence,NaN,NaN,10100.0,ADDEPEV2,RESP046,"(45.5222131, -98.7041456)",10100,10100.0,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,NaN,NaN,NaN,46045.0,0.214,0.004,0.001,0.239,0.504,0.007,0.000,0.058,1.000,0.289,2994.535,2152.445,0.168,0.198,0.162,0.072,Edmunds County,1.0,45.0,0.085,0.724,0.216,0.839,NaN,0.0,NaN,0.875000,53372.0,0.000000,27.8,0.303,1.6,0.113,2.8,3935.0,940.465,5900.0,55.6724,0.000252,-3967.0,3967.000000,NaN,0.621,SD,46.0,NaN,842.090,865.420615,503.081880,940.465,152.355330,67.713480,646.819560,1983.240,0.00000,0.000000,1.093930,228.23,907.344105,209.866667,338.382455,367.266667,232.165000,0.000010,3935.000,NaN,424.031665,NaN,0.034,0.197,NaN,2011.0
2,2011,10420.0,"Akron, OH Metropolitan Statistical Area",Yes,129.0,16.18,12.24,20.12,50.0,%,Crude Prevalence,NaN,NaN,10420.0,ADDEPEV2,RESP046,"(41.1466315, -81.3501066)",10420,10420.0,184.0,"Akron, OH",Metropolitan Statistical Area,NaN,NaN,0.205128,39.0,703200.0,39133.0,0.126,0.002,0.011,0.211,0.512,0.010,0.000,0.010,0.412,0.288,124291.170,104442.390,0.246,0.224,0.133,0.286,Portage County,1.0,133

In [106]:
data.shape

(10259, 100)

In [107]:
data.groupby(['Year']).size()

Year
2011    985
2012    904
2013    873
2014    826
2015    825
2016    854
2017    840
2018    844
2019    859
2020    807
2021    812
2022    830
dtype: int64

In [108]:
data.groupby(['Year', 'cbsacode']).size()

Year  cbsacode
2011  10100       2
      10420       2
      10500       4
      10580       5
      10740       4
                 ..
2022  48140       1
      48300       2
      48620       4
      49340       1
      49660       2
Length: 1582, dtype: int64

In [109]:
data.columns

Index(['Year', 'Locationabbr', 'Locationdesc', 'Response', 'Sample_Size',
       'Data_value', 'Confidence_limit_Low', 'Confidence_limit_High',
       'Display_order', 'Data_value_unit', 'Data_value_type',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'LocationID',
       'QuestionID', 'RESPONSEID', 'GeoLocation', 'cbsacode',
       'metropolitandivisioncode', 'csacode', 'cbsatitle',
       'metropolitanmicropolitanstatis', 'depression_twitter19',
       'CENSUS2019POP', 'depression_twitter10', 'sample_size_twitter10',
       'CENSUS2010POP', 'fipscode', '%_65_and_older',
       '%_american_indian_&_alaska_native', '%_asian',
       '%_below_18_years_of_age', '%_female', '%_hispanic',
       '%_native_hawaiian/other_pacific_islander',
       '%_not_proficient_in_english', '%_rural', 'adult_obesity',
       'adult_population', 'adult_population18_65', 'adult_smoking',
       'children_eligible_for_free_or_reduced_price_lunch',
       'children_in_poverty', 'children_in_sin

In [110]:
data.shape

(10259, 100)

In [111]:

# change end
columns_to_keep = [
    'Locationabbr', 'year','Locationdesc',
    'Sample_Size', 'Data_value', 'Data_value_unit', 'Data_value_type',
     'fipscode', 'population',
    #'adult_population', 'adult_population18_65',  'total_65_and_older',
    'total_adult_obesity', 'total_adult_smoking', #'total_below_18_years_of_age',
    'total_children_in_poverty', 'total_children_in_single-parent_households',
    'total_excessive_drinking', 'total_homicides', #'total_female', 
    'total_mental_health_providers', 'total_motor_vehicle_crash_deaths',
    'total_not_proficient_in_english', 'total_physical_inactivity',
    'total_poor_mental_health_days', 'total_poor_or_fair_health',
    'total_poor_physical_health_days', 'total_premature_death',
    'total_primary_care_physicians', 'total_rural',
    'total_sexually_transmitted_infections', 'total_uninsured_adults',
    'total_violent_crime',"cbsacode", "metropolitandivisioncode","csacode", "cbsatitle", "metropolitanmicropolitanstatis", "depression_twitter19", "depression_twitter10","CENSUS2010POP","CENSUS2019POP"
]
## change end
selected_df = data[columns_to_keep]
selected_df.to_csv(f"{data_dir}20241025selected_smart_chr_cbsa.csv", index=False)


In [112]:
selected_df.head()

,Locationabbr,year,Locationdesc,Sample_Size,Data_value,Data_value_unit,Data_value_type,fipscode,population,total_adult_obesity,total_adult_smoking,total_children_in_poverty,total_children_in_single-parent_households,total_excessive_drinking,total_homicides,total_mental_health_providers,total_motor_vehicle_crash_deaths,total_not_proficient_in_english,total_physical_inactivity,total_poor_mental_health_days,total_poor_or_fair_health,total_poor_physical_health_days,total_premature_death,total_primary_care_physicians,total_rural,total_sexually_transmitted_infections,total_uninsured_adults,total_violent_crime,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,depression_twitter19,depression_twitter10,CENSUS2010POP,CENSUS2019POP
0,10100.0,2011.0,"Aberdeen, SD Micropolitan Statistical Area",88.0,16.75,%,Crude Prevalence,46013.0,35204.0,8058.899680,5217.796064,993.597696,1955.652608,5545.615712,0.704080,0.000080,4.013256,352.040,7594.488512,2816.320000,3442.106304,3520.400000,1823.250364,0.000332,10525.996,79.103388,2899.049400,NaN,10100,10100.0,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,NaN,NaN
1,10100.0,2011.0,"Aberdeen, SD Micropolitan Statistical Area",88.0,16.75,%,Crude Prevalence,46045.0,3935.0,865.420615,503.081880,152.355330,67.713480,646.819560,0.000000,0.000000,1.093930,228.230,907.344105,209.866667,338.382455,367.266667,232.165000,0.000010,3935.000,NaN,424.031665,NaN,10100,10100.0,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,NaN,NaN
2,10420.0,2011.0,"Akron, OH Metropolitan Statistical Area",129.0,16.18,%,Crude Prevalence,39133.0,157530.0,35795.856960,30575.627820,4420.764390,9506.305380,16033.560930,2.835540,0.000805,18.746070,1575.300,33310.033560,15227.900000,14417.775720,16803.200000,10462.197420,0.000754,64902.360,255.513660,15457.473720,NaN,10420,10420.0,184.0,"Akron, OH",Metropolitan Statistical Area,NaN,0.205128,703200.0,NaN
3,10420.0,2011.0,"Akron, OH Metropolitan Statistical Area",129.0,16.18,%,Crude Prevalence,39153.0,542405.0,117246.264800,93797.011840,22136.632860,40192.210500,74535.125480,26.035440,0.003743,49.358855,9763.290,103427.955020,66896.616667,54854.502460,57856.533333,38850.842935,0.005780,23323.415,1977.608630,47423.011555,NaN,10420,10420.0,184.0,"Akron, OH",Metropolitan Statistical Area,NaN,0.205128,703200.0,NaN
4,NaN,2011.0,NaN,NaN,NaN,NaN,NaN,13095.0,95859.0,22906.274922,14615.046576,9188.372727,14793.536034,8923.610169,7.860438,0.000141,15.529158,1150.308,21149.658747,9905.430000,15879.810222,14059.320000,10283.370084,0.001106,12845.106,1031.155263,13208.315751,NaN,10500,NaN,NaN,NaN,NaN,NaN,0.266667,153857.0,NaN


In [114]:
## change
sum_cols = [
    'population',  # 'adult_population', 'adult_population18_65', 'total_65_and_older',
    'total_adult_obesity', 'total_adult_smoking',  # 'total_below_18_years_of_age',
    'total_children_in_poverty', 'total_children_in_single-parent_households',
    'total_excessive_drinking', 'total_homicides',  # 'total_female',
    'total_mental_health_providers', 'total_motor_vehicle_crash_deaths',
    'total_not_proficient_in_english', 'total_physical_inactivity',
    'total_poor_or_fair_health', 'total_premature_death',
    'total_primary_care_physicians', 'total_rural',
    'total_sexually_transmitted_infections', 'total_uninsured_adults',
    'total_violent_crime'
]

avg_cols = ['total_poor_physical_health_days', 'total_poor_mental_health_days']


def calculate_weighted(group):
    result = {}

    total_population = group['population'].fillna(0).sum()
    
    if total_population == 0: # if total_population is 0, use the first non-NaN Twitter population data as a fallback
        total_population = group['CENSUS2019POP'].dropna().iloc[0] if not group['CENSUS2019POP'].dropna().empty else \
                           group['CENSUS2010POP'].dropna().iloc[0] if not group['CENSUS2010POP'].dropna().empty else 0

    for col in sum_cols:
        weighted_sum = (group[col].fillna(0)).sum()
        sum_population_non_na = group.loc[group[col].notna(), 'population'].fillna(0).sum()
        result[col] = (weighted_sum / sum_population_non_na) * total_population if sum_population_non_na != 0 else 0

    for col in avg_cols:
        non_na_values = group[col].dropna()
        result[col] = non_na_values.mean() if not non_na_values.empty else 0

    result['depression_twitter19'] = group['depression_twitter19'].dropna().iloc[0] if not group['depression_twitter19'].dropna().empty else None
    result['depression_twitter10'] = group['depression_twitter10'].dropna().iloc[0] if not group['depression_twitter10'].dropna().empty else None
    result['population'] = group['population'].sum()
    result['CENSUS2019POP'] = group['CENSUS2019POP'].dropna().iloc[0] if not group['CENSUS2019POP'].dropna().empty else None
    result['CENSUS2010POP'] = group['CENSUS2010POP'].dropna().iloc[0] if not group['CENSUS2010POP'].dropna().empty else None

    return pd.Series(result)

aggregated_df = selected_df.groupby([
    'year', "cbsacode", 'Sample_Size', 'Data_value', 'Data_value_unit', 'Data_value_type'
]).apply(calculate_weighted).reset_index()

## change end

aggregated_df['year'] = aggregated_df['year'].astype(int)
print(aggregated_df.shape)

(1605, 30)


In [115]:
aggregated_df.head()

,year,cbsacode,Sample_Size,Data_value,Data_value_unit,Data_value_type,population,total_adult_obesity,total_adult_smoking,total_children_in_poverty,total_children_in_single-parent_households,total_excessive_drinking,total_homicides,total_mental_health_providers,total_motor_vehicle_crash_deaths,total_not_proficient_in_english,total_physical_inactivity,total_poor_or_fair_health,total_premature_death,total_primary_care_physicians,total_rural,total_sexually_transmitted_infections,total_uninsured_adults,total_violent_crime,total_poor_physical_health_days,total_poor_mental_health_days,depression_twitter19,depression_twitter10,CENSUS2019POP,CENSUS2010POP
0,2011,10100,88.0,16.75,%,Crude Prevalence,39139.0,8924.320295,5720.877944,1145.953026,2023.366088,6192.435272,0.704080,0.000080,5.107186,580.270,8501.832617,3780.488759,2055.415364,0.000342,14460.996,87.945333,3323.081065,0.000000,1943.833333,1513.093333,NaN,NaN,NaN,NaN
1,2011,10420,129.0,16.18,%,Crude Prevalence,699935.0,153042.121760,124372.639660,26557.397250,49698.515880,90568.686410,28.870980,0.004548,68.104925,11338.590,136737.988580,69272.278180,49313.040355,0.006535,88225.775,2233.122290,62880.485275,0.000000,37329.866667,41062.258333,NaN,0.205128,NaN,703200.0
2,2011,10740,720.0,21.78,%,Crude Prevalence,857903.0,135700.693621,123687.787094,41015.780399,76095.770766,85547.993919,69.152880,0.001758,149.082893,73774.102,114322.329054,97397.000813,67685.000001,0.009477,87515.802,3814.313737,146799.069585,0.000000,26900.990000,25148.296667,NaN,NaN,NaN,NaN
3,2011,10900,190.0,20.59,%,Crude Prevalence,816012.0,174990.517134,135992.901739,24974.250197,52683.321229,108036.992052,0.000000,0.003242,103.386404,40602.136,158619.859917,87644.863457,52149.148954,0.007402,157443.539,2027.464992,65794.536879,2208.332807,25304.955833,24480.254167,NaN,NaN,NaN,NaN
4,2011,11260,241.0,17.54,%,Crude Prevalence,374553.0,74071.419735,58399.185228,10178.969077,28209.733586,49694.181724,25.309652,0.001502,55.901125,17944.814,60960.590914,36882.397352,27108.017442,0.003629,68165.241,2549.323455,54654.508776,0.000000,21961.161667,19316.843333,NaN,NaN,NaN,NaN


In [ ]:
aggregated_df.groupby("year").size()

In [ ]:
aggregated_df.groupby(["year", "cbsacode"]).size()

## Save into csv

In [31]:
## change
aggregated_df.sort_values(by=['cbsacode', 'year'], ascending=[True, True]).to_csv(f"{data_dir}20241030agg_smart_chr.csv", index=False)

In [ ]:
aggregated_df.head(3)
